# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

import numpy as np
import pandas as pd

import sys
import os
import re
from sqlalchemy import create_engine
import pickle

from scipy.stats import gmean
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator,TransformerMixin

[nltk_data] Downloading package punkt to /Users/kcen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/kcen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/kcen/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
database_filepath = "../data/disaster_response_db.db"
engine = create_engine('sqlite:///' + database_filepath)
table_name = os.path.basename(database_filepath).replace(".db","") + "_table"
df = pd.read_sql_table(table_name,engine)

df = df.loc[:,~((df==1).all()|(df==0).all())]
df['related']=df['related'].map(lambda x: 1 if x == 2 else 0)

X = df['message']
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text, url_placeholder_str="urlplaceholder"):
    """
    INPUT:
        text: text that needs to be tokenized
    OUTPUT:
        clean_tokens: list of tokens extracted
    """
    
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    
    for detected_url in detected_urls:
        text = text.replace(detected_url, url_placeholder_str)

    # Extract the word tokens from the provided text
    tokens = nltk.word_tokenize(text)
    
    #Lemmanitizer to remove inflectional and derivationally related forms of a word
    lemmatizer = nltk.WordNetLemmatizer()

    clean_tokens = [lemmatizer.lemmatize(w).lower().strip() for w in tokens]
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ]))
            
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline_fitted = pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_prediction_train = pipeline_fitted.predict(X_train)
y_prediction_test = pipeline_fitted.predict(X_test)

print(classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.28      0.11      0.15        47
               request       0.79      0.55      0.64      1139
                 offer       0.00      0.00      0.00        32
           aid_related       0.76      0.59      0.66      2693
          medical_help       0.62      0.28      0.39       514
      medical_products       0.67      0.27      0.39       329
     search_and_rescue       0.51      0.17      0.25       177
              security       0.36      0.08      0.13       105
              military       0.60      0.37      0.46       209
                 water       0.71      0.63      0.67       439
                  food       0.81      0.70      0.75       724
               shelter       0.77      0.51      0.61       588
              clothing       0.72      0.44      0.55       106
                 money       0.62      0.35      0.45       150
        missing_people       0.67      

In [8]:
print(classification_report(y_train.values, y_prediction_train, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.54      0.18      0.28       141
               request       0.80      0.55      0.65      3335
                 offer       0.33      0.07      0.12        86
           aid_related       0.77      0.61      0.68      8167
          medical_help       0.68      0.29      0.40      1570
      medical_products       0.70      0.34      0.46       984
     search_and_rescue       0.65      0.23      0.34       547
              security       0.41      0.08      0.14       366
              military       0.66      0.33      0.44       651
                 water       0.77      0.67      0.71      1233
                  food       0.80      0.69      0.74      2199
               shelter       0.80      0.55      0.65      1726
              clothing       0.74      0.43      0.54       299
                 money       0.64      0.35      0.45       454
        missing_people       0.70      

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
parameters = {'classifier__estimator__learning_rate': [0.01, 0.02, 0.05],
              'classifier__estimator__n_estimators': [10, 20, 40]}

cv = GridSearchCV(pipeline, param_grid=parameters, scoring='f1_micro', n_jobs=-1)
cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('count_vectorizer',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x7f9b23bbbc10>)),
                                                                                        ('tfidf_transformer',
                                                                                         TfidfTransformer())]))])),
                                       ('classifier',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             n_jobs=-1,
             param_grid={'classifier__estimator__learning_rate': [0.01, 0.02,
                                                                  0.05],
                         'classifier__est

In [11]:
# Get the prediction values from the grid search cross validator
y_prediction_test = cv.predict(X_test)
y_prediction_train = cv.predict(X_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
print(classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.00      0.00      0.00        47
               request       0.60      0.40      0.48      1139
                 offer       0.00      0.00      0.00        32
           aid_related       0.81      0.18      0.30      2693
          medical_help       0.58      0.10      0.17       514
      medical_products       0.69      0.09      0.16       329
     search_and_rescue       0.57      0.16      0.25       177
              security       0.00      0.00      0.00       105
              military       0.50      0.14      0.22       209
                 water       0.59      0.84      0.69       439
                  food       0.77      0.66      0.71       724
               shelter       0.90      0.29      0.44       588
              clothing       0.50      0.01      0.02       106
                 money       0.00      0.00      0.00       150
        missing_people       0.69      

In [13]:
print(classification_report(y_train.values, y_prediction_train, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.00      0.00      0.00       141
               request       0.62      0.37      0.46      3335
                 offer       1.00      0.02      0.05        86
           aid_related       0.80      0.19      0.31      8167
          medical_help       0.65      0.12      0.20      1570
      medical_products       0.74      0.14      0.24       984
     search_and_rescue       0.65      0.20      0.31       547
              security       0.00      0.00      0.00       366
              military       0.53      0.14      0.22       651
                 water       0.57      0.85      0.68      1233
                  food       0.77      0.68      0.73      2199
               shelter       0.83      0.30      0.44      1726
              clothing       1.00      0.00      0.01       299
                 money       1.00      0.01      0.02       454
        missing_people       0.67      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [14]:
# Build a custom transformer which extracts the first verb of a sentence
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    This class extract the first verb of a sentence and creates a new feature for the ML classifier
    """

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

pipelineWithVerbTrans = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ])),

            ('starting_verb_transformer', StartingVerbExtractor())
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])


In [15]:

#Use pipelineWithVerbTrans which includes StartingVerbEstimator
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline_fitted = pipelineWithVerbTrans.fit(X_train, y_train)

y_prediction_train = pipeline_fitted.predict(X_train)
y_prediction_test = pipeline_fitted.predict(X_test)

# Print classification report on test data
print(classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.42      0.10      0.16        50
               request       0.77      0.53      0.63      1104
                 offer       0.00      0.00      0.00        28
           aid_related       0.75      0.59      0.66      2695
          medical_help       0.65      0.29      0.40       510
      medical_products       0.58      0.29      0.38       327
     search_and_rescue       0.57      0.19      0.28       167
              security       0.13      0.03      0.04       119
              military       0.49      0.32      0.39       192
                 water       0.76      0.62      0.69       445
                  food       0.79      0.69      0.74       715
               shelter       0.77      0.53      0.63       564
              clothing       0.80      0.39      0.53       109
                 money       0.65      0.35      0.45       159
        missing_people       0.50      

In [16]:
print(classification_report(y_train.values, y_prediction_train, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.68      0.19      0.30       138
               request       0.80      0.56      0.66      3370
                 offer       0.56      0.11      0.19        90
           aid_related       0.78      0.61      0.68      8165
          medical_help       0.64      0.28      0.39      1574
      medical_products       0.72      0.35      0.47       986
     search_and_rescue       0.68      0.23      0.34       557
              security       0.40      0.07      0.11       352
              military       0.69      0.35      0.47       668
                 water       0.78      0.65      0.71      1227
                  food       0.82      0.69      0.75      2208
               shelter       0.79      0.55      0.65      1750
              clothing       0.78      0.47      0.59       296
                 money       0.63      0.30      0.40       445
        missing_people       0.79      

### 9. Export your model as a pickle file

In [17]:
m = pickle.dumps('classifier.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.